# Imports

In [ ]:
%load_ext lab_black

import h5py
import os

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm

from typing import Dict, List, Optional, Tuple
import numpy as np
import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    cnn,
)

colors = pc.qualitative.D3
colors_seq = pc.sequential.dense
angstrom = "\u212B"

In [ ]:
def sum_timescans(files, ddir, new_filename):
    datas = []
    for i in range(0, len(files)):
        ad = ARPES_DATA[files[i]]
        datas.append(ad.data)
    data_avg = np.sum(datas, axis=0)
    print(data_avg.shape)

    new_data = data_avg

    new_fn = os.path.join(ddir, new_filename)

    with h5py.File(
        new_fn, "w"
    ) as f:  # Note: 'w' creates a new empty file (or overwrites), use 'r+' to modify an existing file
        f["data"] = new_data.T
        axes_names = [
            "angles",
            "energies",
        ]  # Change these to match your axes labels
        axes = [ad.theta, ad.energy]
        for axis, name in zip(axes, axes_names):
            f[name] = np.atleast_2d(axis).T
        entry_group = f.require_group("entry1")
        entry_group["ScanValues"] = np.atleast_2d(ad.phi_or_time).T
    return new_fn

# Load Data

In [ ]:
ddir = r"E:\atully\arpes_data\2023_June\C60\ARPES\TR"

# M to K

In [ ]:
files = []
files = [f"TR4_Ali_avg.h5"]  # CT2: -1 to 1 ps

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_4 = ARPES_DATA[files[0]]

In [ ]:
files = []
files = [f"TR6_Ali_avg.h5"]  # CT2: -0.5 to 0.5 ps

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_6 = ARPES_DATA[files[0]]

In [ ]:
files = []
files = [
    f"TR7_Ali_avg.h5"
]  # CT1: -1 to 1 ps in steps of 100 fs, then out to 30 ps with variable steps

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_7 = ARPES_DATA[files[0]]

# G to K

In [ ]:
files = []
files = [f"TR9&TR11_Ali_avg.h5"]  # CT2: -0.5 to 0.5 ps

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_9_11 = ARPES_DATA[files[0]]

In [ ]:
files = []
files = [
    f"TR12_Ali_avg_best.h5"
]  # CT1: -1 to 1 ps in steps of 100 fs, then out to 30 ps with variable steps --> best because only first 45 cycles before cone shows up

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_12 = ARPES_DATA[files[0]]

# G

# Feb Comparison

In [ ]:
ddir = r"E:\atully\arpes_data\2023_February\6eV\TR"
files = []
files = [
    "TR4_Ali_avg.h5"
]  # 2.6 eV center energy; -1 to 1 ps, same number of steps as first 2 ps of TR3

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_feb_4 = ARPES_DATA[files[0]]

In [ ]:
ddir = r"E:\atully\arpes_data\2023_February\6eV\TR"
files = ["TR3_Ali_avg.h5"]  # 2.15 eV center energy; -1 to 2 ps

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_feb_3 = ARPES_DATA[files[0]]

# Test

In [ ]:
ad = ad_4
ad = ad_6
ad = ad_9_11

for k in ["energy", "theta", "phi_or_time"]:
    print(f"{k}.shape = {getattr(ad, k).shape}")
print(f"Data.shape = {ad.data.shape}")

In [ ]:
print(f"Delay range (mm): {np.min(ad.phi_or_time), np.max(ad.phi_or_time)}")
print(
    f"Energy range (eV): {np.round(np.min(ad.energy), 2), np.round(np.max(ad.energy), 2)}"
)
print(f"Theta range: {np.round(np.min(ad.theta), 1), np.round(np.max(ad.theta), 1)}")

# Analysis

In [ ]:
## Zero Delay ##

time_zero = 34.8225  # Bi2Se3

time_zero_feb = 37.958  # Feb (from BiSe)

## HOMO is at 2.05 eV below EF, based on fits from this data averaged with fits from tr-ARPES results ##

EF_400 = 1.91  # in kinetic energy, slit 400
EF_700 = 1.94  # in kinetic energy, slit 700

homo = -2.05

homo_400 = homo + EF_400
homo_700 = homo + EF_700

In [ ]:
## Integrate over desired angular range ##

slice_dim = "x"
slice_val = 0
int_range = 20  # if this value is more that the integration range, my get_2D_slice function will just integrate over the max range.

xlim = None
ylim = None
x_bin = 1
y_bin = 1

In [ ]:
all_vals = []
for ad in [ad_4, ad_6, ad_7, ad_9_11, ad_12, ad_feb_4, ad_feb_3]:
    all_vals.append(
        tr_functions.slice_datacube(
            ad_dataclass=ad,
            slice_dim=slice_dim,
            slice_val=slice_val,
            int_range=int_range,
            xlim=xlim,
            ylim=(
                ad.energy[57],
                ad.energy[1007],
            ),  # get rid of zero padding on datasets
            x_bin=x_bin,
            y_bin=y_bin,
            norm_data=False,
            plot_data=False,
        )
    )

## M to K ##
x_4, y_4, d_4 = all_vals[0]
x_6, y_6, d_6 = all_vals[1]
x_7, y_7, d_7 = all_vals[2]

## G to K ##
x_9_11, y_9_11, d_9_11 = all_vals[3]
x_12, y_12, d_12 = all_vals[4]

## Feb ##
feb_x_4, feb_y_4, feb_d_4 = all_vals[5]
feb_x_3, feb_y_3, feb_d_3 = all_vals[6]

In [ ]:
## Plot Data: MPL ##

fig, ax = plt.subplots(1)

## G to K ##
# ax.pcolormesh(x_9_11, y_9_11, d_9_11, shading="auto", cmap="plasma")
# ax.pcolormesh(x_12, y_12, d_12, shading="auto", cmap="plasma")

## M to K ##
ax.pcolormesh(x_4, y_4, d_4, shading="auto", cmap="plasma")  # vmin=0, vmax=0.1
ax.pcolormesh(x_6, y_6, d_6, shading="auto", cmap="plasma")  # vmin=0, vmax=2.5
ax.pcolormesh(x_7, y_7, d_7, shading="auto", cmap="plasma")  # vmin=0, vmax=0.2

ax.set_xlim(xmin=34.758, xmax=34.905)

# plt.save_fig(r'C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR3&TR4&T11_plasma_mpl.png')

In [ ]:
## Adjust energy axis to be relative to HOMO ##

homo_zero = False
homo_zero = True

if homo_zero:
    x4, y4, d4 = x_4, y_4 - homo_400, d_4
    x6, y6, d6 = x_6, y_6 - homo_400, d_6
    x7, y7, d7 = x_7, y_7 - homo_400, d_7
    x11, y11, d11 = x_9_11, y_9_11 - homo_400, d_9_11
    x12, y12, d12 = x_12, y_12 - homo_400, d_12

    # Feb
    feb_x4, feb_y4, feb_d4 = feb_x_4, feb_y_4 - homo_400, feb_d_4
    feb_x3, feb_y3, feb_d3 = feb_x_3, feb_y_3 - homo_700, feb_d_3

# Plot Data

In [ ]:
## Plot Data ##

In [ ]:
yaxis_title = "E - E<sub>HOMO</sub> (eV)"
xaxis_title = "Delay (ps)"

In [ ]:
## Plot CT2 ##

## M - K ##
x_plot, y_plot, z_plot = x4, y4, analysis_functions.norm_data(d4)
x_plot, y_plot, z_plot = x6, y6, analysis_functions.norm_data(d6)
title = f"M - K"
time_zero_plot = time_zero

## G - K ##
x_plot, y_plot, z_plot = x11, y11, analysis_functions.norm_data(d11)
title = f"G - K"
time_zero_plot = time_zero

## Feb: G ##
# x_plot, y_plot, z_plot = feb_x4, feb_y4, analysis_functions.norm_data(feb_d4)
# title = f"G"
# time_zero_plot = time_zero_feb

## toggle_time ?
toggle_time = "picoseconds"
# toggle_time = "mm"

## Logplot?
logplot = False
# logplot = True

if logplot:
    z_plot = np.log(z_plot)
    title = f"{title} (logplot)"
else:
    title = title

## Convert mm to ps
if toggle_time == "picoseconds":
    x_plot = tr_functions.mm_to_ps(x_plot, time_zero_plot)

## Plot Data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    dtick_y=0.2,
)
fig.add_trace(
    go.Heatmap(x=x_plot, y=y_plot, z=z_plot, coloraxis="coloraxis")
    # np.log(data)
)

fig.update_coloraxes(cmin=0, cmax=1)
# fig.update_coloraxes(colorscale="hot", cmin=None, cmax=0.3, reversescale=True)
fig.update_layout(width=800, height=600)
fig.show()

# fig.write_image(r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR3&TR4.png")

In [ ]:
## EDCs with different time integration ##

# title = f"{files[0]}"
title = title
spacing = 0

x_plot, y_plot, z_plot = x_plot, y_plot, z_plot

if np.min(x_plot) > 0:
    x_edc = tr_functions.mm_to_ps(x_plot, time_zero)
else:
    x_edc = x_plot


## Normalize?
normalize = False
# normalize = True

## Logplot?
logplot = False
# logplot = True

if logplot:
    title = f"{title}<br>(logplot)"
else:
    title = f"{title}"


## Set up integration limits
# xlim1 = (-2, -1)
# xlim2 = (-0.5, 0.5)
# xlim3 = (1, 2)

# xlim1 = (-0.5, 0)
# xlim2 = (-0.25, 0.25)
# xlim3 = (0, 0.5)
# xlims = [xlim1, xlim2, xlim3]

xlim1 = (-0.5, 0.5)
xlims = [xlim1]

# xlim = None
ylim = None


# Get and plot data

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
)

for i, xlim in enumerate(xlims):
    color = colors[i % len(colors)]

    y_1d, col = tr_functions.get_1d_x_slice(
        x=x_edc,
        y=y_plot,
        data=z_plot,
        ylims=ylim,
        x_range=xlim,
    )

    if normalize:
        col = analysis_functions.norm_data(col)

    if logplot:
        col = np.log(col)

    # Plot Data
    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=i * spacing + col,
            name=f"{np.round(xlim[0], 2), np.round(xlim[1], 2)} ps",
            line=dict(color=color),
        )
    )

fig.show()

In [ ]:
x = y_1d
data = col

offset_type = "constant"
plot_fit = True


## Offset
c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

## Gaussians
gauss1 = fitting_functions.make_gaussian(num="A_", amplitude=1, center=2.5, sigma=0.1)
gauss2 = fitting_functions.make_gaussian(num="B_", amplitude=1, center=2.75, sigma=0.1)
gauss3 = fitting_functions.make_gaussian(num="C_", amplitude=1, center=2.95, sigma=0.1)


## Full model
full_model = gauss1 + offset
full_model = gauss1 + gauss2 + gauss3 + offset

params = full_model.make_params()

# params["iA__center"].max = 2.57
# params["iA__fwhm"].max = 0.4
# params["iB__fwhm"].min = 0.1
# params["iC__fwhm"].min = 0.1


fit = full_model.fit(data, x=x, params=params)
if plot_fit:
    fit.plot()

print(f"Center A = {fit.params['iA__center'].value:.2f} eV")
print(f"FWHM A = {fit.params['iA__fwhm'].value:.3f} eV")
print(f"Center B = {fit.params['iB__center'].value:.2f} eV")
print(f"FWHM B = {fit.params['iB__fwhm'].value:.3f} eV")
print(f"Center C = {fit.params['iC__center'].value:.2f} eV")
print(f"FWHM C = {fit.params['iC__fwhm'].value:.3f} eV")

In [ ]:
# fit.params

In [ ]:
## Plot Data and Fit
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    dtick_y=0.1,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x_edc,
    y=y_plot,
    data=z_plot,
    ylims=ylim,
    x_range=xlim,
)

fig.add_trace(
    go.Scatter(
        x=y_1d,
        y=col,
        name=f"{np.round(xlim[0], 2), np.round(xlim[1], 2)} ps",
        line=dict(color=colors[0]),
    )
)

fig.add_trace(go.Scatter(x=y_1d, y=fit.eval(x=y_1d), name="fit"))

fig.update_layout(showlegend=True)

components = fit.eval_components(x=y_1d)
for model_name, model_value in list(components.items())[0:3]:
    fig.add_annotation(
        x=fit.params[f"{model_name}center"].value,
        y=fit.eval(x=fit.params[f"{model_name}center"].value),
        text=f'{fit.params[f"{model_name}center"].value:.2f} ',
        font=dict(size=18, color=colors[0]),
        # ax=-300,
        ay=-60,
        bgcolor="white",
        # opacity=1,
        # bordercolor=colors[0],
        # borderwidth=2,
        # borderpad=4,
    )

fig.show()

# CT1

In [ ]:
## Plot CT1 ##

xlim = (-1, 1.1)
# xlim = None
ylim = None

## M - K ##
x_plot, y_plot, z_plot = x7, y7, analysis_functions.norm_data(d7)
title = f"M - K"
time_zero_plot = time_zero

## G - K ##
# x_plot, y_plot, z_plot = x12, y12, analysis_functions.norm_data(d12)
# title = f"G - K"
# time_zero_plot = time_zero

# Feb: G ##
x_plot, y_plot, z_plot = feb_x3, feb_y3, analysis_functions.norm_data(feb_d3)
x_plot, y_plot, z_plot = feb_x3, feb_y3, feb_d3
title = f"G"
time_zero_plot = time_zero_feb

## toggle_time ?
toggle_time = "picoseconds"
# toggle_time = "mm"

## Logplot?
logplot = False
# logplot = True

if logplot:
    z_plot = np.log(z_plot)
    title = f"{title} (logplot)"
else:
    title = title

## Convert mm to ps
if toggle_time == "picoseconds":
    x_plot = tr_functions.mm_to_ps(x_plot, time_zero_plot)

x_plot, y_plot, z_plot = analysis_functions.limit_dataset(
    x_plot, y_plot, z_plot, xlim=xlim, ylim=ylim
)

## Plot Data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    dtick_y=0.2,
)
fig.add_trace(
    go.Heatmap(x=x_plot, y=y_plot, z=z_plot, coloraxis="coloraxis")
    # np.log(data)
)

fig.update_coloraxes(cmin=0, cmax=1)
# fig.update_coloraxes(colorscale="hot", cmin=None, cmax=0.3, reversescale=True)
fig.update_layout(width=800, height=600)
fig.show()

In [ ]:
## EDCs with different time integration ##

title = title

x_plot, y_plot, z_plot = x_plot, y_plot, z_plot

## Ensure xaxis in ps
if np.min(x_plot) > 0:
    x_edc = tr_functions.mm_to_ps(x_plot, time_zero)
else:
    x_edc = x_plot

## Logplot?
logplot = False
# logplot = True

if logplot:
    # z_plot = np.log(z_plot)
    title = f"{title}<br>(logplot)"
else:
    title = f"{title}"

## Set up integration limits
xlim1 = (
    np.min(x_edc),
    -0.5,
)  # purely negative delay (taking into account time overlap pulsewidth)
xlim2 = (-0.5, 0.5)  # zero delay
xlim3 = (0.5, 1)  # longer pumped signal


xlims = [xlim1, xlim2, xlim3]
ylim = None

## Get and plot data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=yaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

for i, xlim in enumerate(xlims):
    color = colors[i % len(colors)]

    y_1d, col = tr_functions.get_1d_x_slice(
        x=x_edc,
        y=y_plot,
        data=z_plot,
        ylims=ylim,
        x_range=xlim,
    )

    if logplot:
        col = np.log(col)

    # Plot Data
    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=col,
            name=f"{np.round(xlim[0], 1), np.round(xlim[1], 2)} ps",
            line=dict(color=color),
        )
    )

fig.show()

# fig.write_image(
#     r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR11&TR3&TR4_EDC.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR11&TR3&TR4_EDC_logplot.png"
# )

# Fitting Peaks

In [ ]:
## EDCs with different time integration ##

title = title
x_plot, y_plot, z_plot = x_plot, y_plot, z_plot

# xlim = (-1, -0.5)
xlim = (0, 1)
ylim = None

logplot = False

# Get and plot data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=yaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x_edc,
    y=y_plot,
    data=z_plot,
    ylims=ylim,
    x_range=xlim,
)

if logplot:
    col = np.log(col)

fig.add_trace(
    go.Scatter(
        x=y_1d,
        y=col,
        name=f"{np.round(xlim[0], 2), np.round(xlim[1], 2)} ps",
        line=dict(color=colors[0]),
    )
)

fig.update_layout(showlegend=True)

fig.show()

In [ ]:
x = y_1d
data = col

offset_type = "linear"
offset_type = "constant"
plot_fit = True


## FD
def fermi_dirac(x, center, theta, amp):
    arg = (x - center) / (2 * theta)  # x=E, center=mu, theta = k_B * T
    return -amp / 2 * np.tanh(arg)


## Offset
c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

## Gaussians
gauss1 = fitting_functions.make_gaussian(num="A_", amplitude=1, center=1.95, sigma=0.1)
gauss2 = fitting_functions.make_gaussian(num="B_", amplitude=1, center=2.15, sigma=0.1)
gauss3 = fitting_functions.make_gaussian(num="C_", amplitude=0.5, center=2.4, sigma=0.1)
# gauss4 = fitting_functions.make_gaussian(num="D_", amplitude=0.5, center=2.5, sigma=0.1)
# gauss5 = fitting_functions.make_gaussian(num="E_", amplitude=0.5, center=2.6, sigma=0.1)


## Full model
full_model = lm.models.Model(fermi_dirac) + gauss1 + gauss2 + gauss3 + offset
# full_model = lm.models.Model(fermi_dirac) + offset

## Run model with linear params starting values, force constant offset
# if offset_type == "constant":
#     params = linear_params.copy()
#     params["b"].value = 0
#     params["b"].vary = False

params = full_model.make_params()

params[
    "center"
].value = 2.0325  # TR12, determined by fitting from (0, 100) ps with a constant offset (also agrees with -1 to -0.5 integration window on tr7)
params["center"].vary = False
T = 10.6  # measurement temp
k_B = 8.617333e-5  # eV/K
params["theta"].value = k_B * (10.6)
params["theta"].min = 0

## Amp initial params
params["amp"].value = 1
params["amp"].min = 0

## Amp value fit from -1 to -0.5 ps
# params["amp"].value = 0.605
# params["amp"].vary = False

params["iA__center"].min = 1.4
params["iA__center"].max = 2.0
params["iB__center"].min = 2.0
params["iB__center"].max = 2.3
params["iC__center"].min = 2.3
params["iC__center"].max = 2.7

# # params["iD__center"].min = 2.45
# # params["iD__center"].max = 2.55
# # params["iE__center"].min = 2.55
# # params["iE__center"].max = 2.7

# params["iA__fwhm"].max = 0.5
# params["iB__fwhm"].max = 0.5
# params["iC__fwhm"].max = 0.5
# # params["iD__fwhm"].max = 0.3
# # params["iE__fwhm"].max = 0.3

fit = full_model.fit(data, x=x, params=params)

if plot_fit:
    fit.plot()

print(f"Center A = {fit.params['iA__center'].value:.2f} eV")
print(f"FWHM A = {fit.params['iA__fwhm'].value:.3f} eV")
print(f"Center B = {fit.params['iB__center'].value:.2f} eV")
print(f"FWHM B = {fit.params['iB__fwhm'].value:.3f} eV")
print(f"Center C = {fit.params['iC__center'].value:.2f} eV")
print(f"FWHM C = {fit.params['iC__fwhm'].value:.3f} eV")

# # print(f"Center D = {fit.params['iD__center'].value:.2f} eV")
# # print(f"FWHM D = {fit.params['iD__fwhm'].value:.3f} eV")
# # print(f"Center E = {fit.params['iE__center'].value:.2f} eV")
# # print(f"FWHM E = {fit.params['iE__fwhm'].value:.3f} eV")


# print(f"FD Center = {fit.params['center'].value:.2f} eV")

In [ ]:
## Plot Data and Fit
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    dtick_y=0.2,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x_edc,
    y=y_plot,
    data=z_plot,
    ylims=ylim,
    x_range=xlim,
)

fig.add_trace(
    go.Scatter(
        x=y_1d,
        y=col,
        name=f"{np.round(xlim[0], 2), np.round(xlim[1], 2)} ps",
        line=dict(color=colors[0]),
    )
)

fig.add_trace(go.Scatter(x=y_1d, y=fit.eval(x=y_1d), name="fit"))

fig.update_layout(showlegend=True)

components = fit.eval_components(x=y_1d)
for model_name, model_value in list(components.items())[1:4]:
    fig.add_annotation(
        x=fit.params[f"{model_name}center"].value,
        y=fit.eval(x=fit.params[f"{model_name}center"].value),
        text=f'{fit.params[f"{model_name}center"].value:.2f} ',
        font=dict(size=18, color=colors[0]),
        # ax=-300,
        ay=-60,
        bgcolor="white",
        # opacity=1,
        # bordercolor=colors[0],
        # borderwidth=2,
        # borderpad=4,
    )

fig.show()

In [ ]:
## Plot Fit Components ##

fig = tr_functions.thesis_fig(
    title=f"{title}<br> Fit Components",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

components = fit.eval_components(x=y_1d)
for model_name, model_value in components.items():
    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=model_value,
            name=model_name,
        )
    )

fig.data[4].update(name="offset")

for model_name, model_value in list(components.items())[1:4]:
    fig.add_annotation(
        x=fit.params[f"{model_name}center"].value,
        # y=fit.eval(x=fit.params[f"{model_name}center"].value),
        # xref="x domain",
        yref="y domain",
        # The arrow head will be 25% along the x axis, starting from the left
        # x=0.25,
        # The arrow head will be 40% along the y axis, starting from the bottom
        y=0.3,
        showarrow=False,
        text=f'{fit.params[f"{model_name}center"].value:.2f} eV<br>{fit.params[f"{model_name}fwhm"].value:.2f} eV',
        font=dict(size=12),
        # ax=-300,
        # ay=0,
        bgcolor="white",
        # opacity=1,
        # bordercolor=colors[0],
        # borderwidth=2,
        # borderpad=4,
    )


fig.show()